<a href="https://colab.research.google.com/github/Vaibhavs10/scratchpad/blob/main/bark_vocos_test.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install -q transformers vocos

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.7/7.7 MB 20.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 302.0/302.0 kB 33.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.8/3.8 MB 49.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 31.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.6/44.6 kB 5.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.7/3.7 MB 58.1 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 295.0/295.0 kB 29.8 MB/s eta 0:00:00


In [2]:
!git clone https://huggingface.co/reach-vb/bark-vocos && cp bark-vocos/* .

Cloning into 'bark-vocos'...
remote: Enumerating objects: 10, done.
remote: Counting objects: 100% (7/7), done.
remote: Compressing objects: 100% (7/7), done.
remote: Total 10 (delta 2), reused 0 (delta 0), pack-reused 3
Unpacking objects: 100% (10/10), 16.42 KiB | 2.35 MiB/s, done.


In [3]:
from bark_vocos import BarkModel as BarkVocosFine
from vocos import Vocos

In [4]:
model_class_vocos = BarkVocosFine

In [5]:
model = model_class_vocos.from_pretrained("suno/bark-small",)

Downloading: "https://dl.fbaipublicfiles.com/encodec/v0/encodec_24khz-d7cc33bc.th" to /root/.cache/torch/hub/checkpoints/encodec_24khz-d7cc33bc.th
100%|██████████| 88.9M/88.9M [00:00<00:00, 128MB/s]
Some weights of the model checkpoint at suno/bark-small were not used when initializing BarkModel: ['codec_model.quantizer.layers.13.codebook.embed', 'codec_model.encoder.layers.1.block.1.conv.bias', 'codec_model.quantizer.layers.10.codebook.embed', 'codec_model.encoder.layers.1.block.3.conv.bias', 'codec_model.encoder.layers.4.block.1.conv.bias', 'codec_model.encoder.layers.4.block.3.conv.weight_v', 'codec_model.decoder.layers.7.shortcut.conv.weight_g', 'codec_model.quantizer.layers.23.codebook.embed', 'codec_model.quantizer.layers.9.codebook.embed_avg', 'codec_model.encoder.layers.1.block.3.conv.weight_g', 'codec_model.quantizer.layers.13.codebook.inited', 'codec_model.encoder.layers.15.conv.weight_v', 'codec_model.quantizer.layers.0.codebook.embed', 'codec_model.decoder.layers.1.lstm.wei

In [6]:
model.vocos = Vocos.from_pretrained("hubertsiuzdak/vocos-encodec-24khz-v2").to(model.device)

In [7]:
model = model.to("cuda")

In [8]:
from transformers import BarkProcessor
processor = BarkProcessor.from_pretrained("suno/bark-small")

In [9]:
inputs = processor("Hey how are you", "v2/en_speaker_6").to("cuda")
input_ids = inputs["input_ids"]
history_prompt = inputs.get("history_prompt", None)

In [10]:
_ = model.generate(input_ids = input_ids,
                   history_prompt=history_prompt,
                   attention_mask = inputs["attention_mask"],)

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:10000 for open-end generation.


In [11]:
from IPython.display import Audio

Audio(_.cpu().numpy().squeeze(), rate=24000)